# Tranform all targets to target axis: save transformed files

This notebook:
- transform all calibrated target locations to target-clutter axis
- save outputs to `'data_processed/tracks/targets_cal_transformed/'`
- generate `main_info_append_03.csv`

In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation

## Load cal object info

In [2]:
cal_obj_file = '../pp_utils/point_projections/data/list_calibration_clips.xlsx'
df_cal = pd.read_excel(cal_obj_file)

In [3]:
df_cal

,DATE,SESSION,VIDEO,FRAME_START,FRAME_END,CAL_OBJECT,CAL_VIDEO_PATH,CAL_VIDEO_FILENAME,HULA_HOOP_FLIPPED,HULA_HOOP_POINTS,FRAME_MARKED,FRAME_EXTRACTED,SIDEDNESS_CHECKED,COMMENTS
0,20190625,1,GP011466,9356,10533,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190625_s1_GP011466_9356_10533_hula.mp4,Y,"1,4,6,8,10",432,Y,Y,NaN
1,20190625,2,GP011467,4053,4752,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190625_s2_GP011467_4053_4752_hula.mp4,Y,"1,4,6,8,10",325,Y,Y,NaN
2,20190625,3,--,--,--,--,--,--,--,--,--,--,--,NaN
3,20190626,1,GOPR1469,30132,30959,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190626_s1_GOPR1469_30132_30959_hula.mp4,Y,"1,4,6,8,10",436,Y,Y,NaN
4,20190626,2,GP011470,12382,13135,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190626_s2_GP011470_12382_13135_hula.mp4,N,"1,4,6,8,10",376,Y,Y,NaN
5,20190626,3,GP011471,15043,15616,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190626_s3_GP011471_15043_15616_hula.mp4,Y,"1,4,6,8,10",304,Y,Y,NaN
6,20190627,1,GP011472,9936,10594,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190627_s1_GP011472_9936_10594_hula.mp4,Y,"1,4,6,8,10",344,Y,Y,NaN
7,20190627,2,GP011473,24941,25606,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190627_s2_GP011473_24941_25606_hula.mp4,Y,"1,2,4,6,8,10",292,Y,Y,NaN
8,20190627,3,GP011474,14227,14755,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190627_s3_GP011474_14227_14755_hula.mp4,Y,"1,2,4,6,8,10",285,Y,Y,NaN
9,20190628,1,GP011475,10745,11214,hula,/Volumes/MURI_4TB/clutter_infotaxis_analysis/v...,20190628_s1_GP011475_10745_11214_hula.mp4,Y,"1,2,4,6,8,10",213,Y,Y,NaN


## Set paths

In [4]:
target_path = Path('../data_processed/tracks/targets_cal/')
output_path = Path('../data_processed/tracks/targets_cal_transformed/')

if not output_path.exists():
    output_path.mkdir()

In [5]:
target_files = sorted(list(target_path.glob('*.csv')))

In [6]:
len(target_files)

312

There are more target files than track files, likely due to the some tracks difficult to label. Let's try to match the files by going from the track files.

In [7]:
FILENAME_MATCHER = re.compile('(?P<date>\d{8})_' 
                              + 's(?P<session>\d{1})_'
                              + 't(?P<trial>\d{1,2})_'
                              + 'targets_(?P<cal_obj>\w+).csv')

# Transform all targets

In [8]:
for tar_f in target_files:

    df_targets = pd.read_csv(tar_f, index_col=0)

    # get rotation angle
    xy_targets = df_targets.loc[:, 
                                ['TOP_OBJECT_X', 'TOP_OBJECT_Y', 
                                 'BOTTOM_OBJECT_X', 'BOTTOM_OBJECT_Y']].values.reshape(2,2)

    # vector pointing from bottom to top target
    bot_top_vec = xy_targets[1, :] - xy_targets[0, :]
    bot_top_vec = bot_top_vec / np.linalg.norm(bot_top_vec)
    yaw = np.pi/2 - np.arctan2(bot_top_vec[1], bot_top_vec[0])
    R = Rotation.from_euler('xyz', [0, 0, yaw]).as_matrix()
    
    # rotate the two targets
    xy_targets_trans = xy_targets - xy_targets[1, :]
    xy_targets_rot = np.dot(R, np.hstack((xy_targets_trans, np.ones((xy_targets_trans.shape[0], 1)))).T).T
    
    # update targets dataframe
    df_targets.loc[:, ['TOP_OBJECT_X', 'TOP_OBJECT_Y', 
                   'BOTTOM_OBJECT_X', 'BOTTOM_OBJECT_Y']] = xy_targets_rot[:, :2].flatten()
    
    # save new file
    fname = Path(tar_f).stem + '_transformed.csv'
    print('save new file: %s' % fname)
    df_targets.to_csv(output_path.joinpath(fname))


save new file: 20190625_s1_t1_targets_hula_flip_transformed.csv
save new file: 20190625_s1_t2_targets_hula_flip_transformed.csv
save new file: 20190625_s1_t3_targets_hula_flip_transformed.csv
save new file: 20190625_s1_t4_targets_hula_flip_transformed.csv
save new file: 20190625_s1_t5_targets_hula_flip_transformed.csv
save new file: 20190625_s1_t6_targets_hula_flip_transformed.csv
save new file: 20190625_s2_t1_targets_hula_flip_transformed.csv
save new file: 20190625_s2_t2_targets_hula_flip_transformed.csv
save new file: 20190625_s2_t3_targets_hula_flip_transformed.csv
save new file: 20190625_s2_t4_targets_hula_flip_transformed.csv
save new file: 20190625_s2_t5_targets_hula_flip_transformed.csv
save new file: 20190625_s2_t6_targets_hula_flip_transformed.csv
save new file: 20190625_s2_t7_targets_hula_flip_transformed.csv
save new file: 20190625_s2_t8_targets_hula_flip_transformed.csv
save new file: 20190626_s1_t10_targets_hula_flip_transformed.csv
save new file: 20190626_s1_t1_targets_h

save new file: 20190628_s2_t6_targets_hula_flip_transformed.csv
save new file: 20190628_s2_t7_targets_hula_flip_transformed.csv
save new file: 20190628_s2_t8_targets_hula_flip_transformed.csv
save new file: 20190628_s2_t9_targets_hula_flip_transformed.csv
save new file: 20190628_s3_t10_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t1_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t2_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t3_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t4_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t5_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t6_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t7_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t8_targets_hula_noflip_transformed.csv
save new file: 20190628_s3_t9_targets_hula_noflip_transformed.csv
save new file: 20190701_s1_t10_targets_hula_noflip_transformed.csv
save new file: 2

save new file: 20190702_s3_t6_targets_cross_transformed.csv
save new file: 20190702_s3_t6_targets_hula_noflip_transformed.csv
save new file: 20190702_s3_t7_targets_cross_transformed.csv
save new file: 20190702_s3_t7_targets_hula_noflip_transformed.csv
save new file: 20190702_s3_t8_targets_cross_transformed.csv
save new file: 20190702_s3_t8_targets_hula_noflip_transformed.csv
save new file: 20190702_s3_t9_targets_cross_transformed.csv
save new file: 20190702_s3_t9_targets_hula_noflip_transformed.csv
save new file: 20190703_s1_t10_targets_cross_transformed.csv
save new file: 20190703_s1_t10_targets_hula_noflip_transformed.csv
save new file: 20190703_s1_t1_targets_cross_transformed.csv
save new file: 20190703_s1_t1_targets_hula_noflip_transformed.csv
save new file: 20190703_s1_t2_targets_cross_transformed.csv
save new file: 20190703_s1_t2_targets_hula_noflip_transformed.csv
save new file: 20190703_s1_t3_targets_cross_transformed.csv
save new file: 20190703_s1_t3_targets_hula_noflip_transf

save new file: 20190704_s1_t9_targets_cross_transformed.csv
save new file: 20190704_s1_t9_targets_hula_noflip_transformed.csv
save new file: 20190704_s2_t10_targets_cross_transformed.csv
save new file: 20190704_s2_t10_targets_hula_noflip_transformed.csv
save new file: 20190704_s2_t11_targets_cross_transformed.csv
save new file: 20190704_s2_t11_targets_hula_noflip_transformed.csv
save new file: 20190704_s2_t1_targets_cross_transformed.csv
save new file: 20190704_s2_t1_targets_hula_noflip_transformed.csv
save new file: 20190704_s2_t2_targets_cross_transformed.csv
save new file: 20190704_s2_t2_targets_hula_noflip_transformed.csv
save new file: 20190704_s2_t3_targets_cross_transformed.csv
save new file: 20190704_s2_t3_targets_hula_noflip_transformed.csv
save new file: 20190704_s2_t4_targets_cross_transformed.csv
save new file: 20190704_s2_t4_targets_hula_noflip_transformed.csv
save new file: 20190704_s2_t5_targets_cross_transformed.csv
save new file: 20190704_s2_t5_targets_hula_noflip_tran